In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams["font.family"] = 'DejaVu Sans'
plt.rcParams["font.size"] = '12'
plt.rcParams['image.cmap'] = 'rainbow'

2.2.0
Num GPUs Available:  1


In [2]:
dataset = pd.read_csv('Concrete_Data.csv')

print(dataset.head())

print("Shape of data set:", dataset.shape)

cement   slag  flyash  water  superplas  coarse_agg  fine_agg  age  \
0   540.0    0.0     0.0  162.0        2.5      1040.0     676.0   28   
1   540.0    0.0     0.0  162.0        2.5      1055.0     676.0   28   
2   332.5  142.5     0.0  228.0        0.0       932.0     594.0  270   
3   332.5  142.5     0.0  228.0        0.0       932.0     594.0  365   
4   198.6  132.4     0.0  192.0        0.0       978.4     825.5  360   

   strength  
0     79.99  
1     61.89  
2     40.27  
3     41.05  
4     44.30  
Shape of data set: (1030, 9)


In [3]:
test_normality = lambda x: stats.shapiro(x.fillna(0))[1] < 0.01
normal = pd.DataFrame(dataset)
normal = normal.apply(test_normality)
print(not normal.any())

False


In [4]:
import os
os.environ['PYTHONHASHSEED'] = '0'
seed = 6
import random as rn
rn.seed(seed)
np.random.seed(seed)

tf.random.set_seed(seed)

In [5]:
# from sklearn.model_selection import train_test_split
# train_x, test_x, train_y, test_y = train_test_split(dataset.drop('strength', axis=1), dataset['strength'], test_size=0.2, random_state=42)
# print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

train_dataset = dataset.sample(frac=0.8,random_state=69)
test_dataset = dataset.drop(train_dataset.index)

print(train_dataset.head(), train_dataset.shape)
print(test_dataset.head(), test_dataset.shape)

cement   slag  flyash  water  superplas  coarse_agg  fine_agg  age  \
42    237.5  237.5     0.0  228.0        0.0       932.0     594.0  365   
184   222.4    0.0    96.7  189.3        4.5       967.1     870.3    3   
131   379.5  151.2     0.0  153.9       15.9      1134.3     605.0   28   
550   393.0    0.0     0.0  192.0        0.0       940.6     785.6    7   
808   252.0    0.0     0.0  186.0        0.0      1111.0     784.0    7   

     strength  
42      39.00  
184     11.58  
131     52.20  
550     27.74  
808     11.47   (824, 9)
    cement   slag  flyash  water  superplas  coarse_agg  fine_agg  age  \
9    475.0    0.0     0.0  228.0        0.0       932.0     594.0   28   
19   475.0    0.0     0.0  228.0        0.0       932.0     594.0  180   
24   380.0    0.0     0.0  228.0        0.0       932.0     670.0  365   
37   332.5  142.5     0.0  228.0        0.0       932.0     594.0   90   
39   237.5  237.5     0.0  228.0        0.0       932.0     594.0  180   

    

In [6]:
train_stats = train_dataset.describe()
train_stats.pop('strength')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
cement,824.0,281.377306,104.495567,102.0,194.4,269.40,351.250,540.0
slag,824.0,73.903277,86.092338,0.0,0.0,22.00,142.500,359.4
flyash,824.0,53.997937,63.164905,0.0,0.0,0.00,118.300,195.0
water,824.0,181.990413,21.217147,121.8,164.9,185.70,192.000,247.0
superplas,824.0,6.206311,6.001590,0.0,0.0,6.35,10.325,32.2
coarse_agg,824.0,972.764320,78.171931,801.0,932.0,968.00,1030.000,1145.0
fine_agg,824.0,773.141262,80.462484,594.0,724.3,780.10,824.250,992.6
age,824.0,44.993932,62.216931,1.0,7.0,28.00,56.000,365.0


In [7]:
train_labels = train_dataset.pop('strength')
test_labels = test_dataset.pop('strength')

print(train_labels.head(), train_labels.shape)

42     39.00
184    11.58
131    52.20
550    27.74
808    11.47
Name: strength, dtype: float64 (824,)


In [8]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

print(normed_train_data.head())

cement      slag    flyash     water  superplas  coarse_agg  fine_agg  \
42  -0.419896  1.900247 -0.854872  2.168510  -1.034111   -0.521470 -2.226395   
184 -0.564400 -0.858419  0.676041  0.344513  -0.284310   -0.072460  1.207504   
131  0.939013  0.897835 -0.854872 -1.323949   1.615187    2.066415 -2.089685   
550  1.068205 -0.858419 -0.854872  0.471769  -1.034111   -0.411456  0.154839   
808 -0.281134 -0.858419 -0.854872  0.188979  -1.034111    1.768354  0.134954   

          age  
42   5.143392  
184 -0.674960  
131 -0.273140  
550 -0.610669  
808 -0.610669  


In [9]:
def train_network(structure, activation, optimizer, epochs):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units = structure[1], input_dim = structure[0], activation = activation)) 
    if structure[2] > 0:
        model.add(keras.layers.Dense(units = structure[2], activation = activation))
        
    # Output layer
    model.add(keras.layers.Dense(units = structure[-1], activation = None))
    
    # Compiles the model with parameters
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mae', 'mse'])
    
    # This tells the us training has started, so we know that it's actually running
    print('training... ', end = '')
    
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    
    # This trains the network
    training_stats = model.fit(normed_train_data, train_labels, batch_size = 1, epochs = epochs, validation_split = 0.2, verbose = 0, shuffle = False)
    
    # Results!

#     print('train_acc: %0.3f, test_acc: %0.3f' %(training_stats.history['accuracy'][-1], 
#                                                 model.evaluate(test_x, test_y, verbose = 0)[1]))
    
    # This returns the results and the model for use outside the function
    return training_stats, model

# Plots our evaluations in a line graph to see how they compare
def plot_acc(train_acc, test_acc, title):
    # Plots the training and testing accuracy lines
    training_accuracy, = plt.plot(train_acc, label = 'Training Accuracy')
    testing_accuracy, = plt.plot(test_acc, label = 'Testing Accuracy')
    plt.legend(handles = [training_accuracy, testing_accuracy])
    
    # Plots guide lines along y = 0 and y = 1 to help visualise
    xp = np.linspace(0, train_acc.shape[0] - 1, 10 * train_acc.shape[0])
    plt.plot(xp, np.full(xp.shape, 1), c = 'k', linestyle = ':', alpha = 0.5)
    plt.plot(xp, np.full(xp.shape, 0), c = 'k', linestyle = ':', alpha = 0.5)
    
    plt.xticks(range(0, train_acc.shape[0]), range(1, train_acc.shape[0] + 1))
    plt.ylim(0,1)
    plt.title(title)
    
    plt.show()

# Plots our evaluations in a bar chart to see how they compare
def bar_acc(train_acc, test_acc, title, xticks):
    index = range(1, train_acc.shape[0] + 1)
    
    # Plots the training and testing accuracy bars
    training_accuracy = plt.bar(index, train_acc, 0.4, align = 'center')
    testing_accuracy = plt.bar(index, test_acc, 0.4, align = 'edge')
    plt.legend((training_accuracy[0], testing_accuracy[0]), ('Training Accuracy', 'Testing Accuracy'))
    
    plt.xticks(index, xticks)
    plt.title(title)
    
    plt.show()

In [10]:
for hidden1 in range (256,257):
    print('Evaluating model with %i hidden neurons... ' %hidden1, end = '')

    training_stats, model = train_network(structure = [8, hidden1, hidden1, 1], activation = 'relu', optimizer = 'RMSprop', epochs = 1000)

Evaluating model with 256 hidden neurons...training...

In [11]:
hist = pd.DataFrame(training_stats.history)
hist['epoch'] = training_stats.epoch
hist.tail()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
995,9.278429,2.147724,9.278429,63.672279,5.841142,63.672279,995
996,9.718360,2.257344,9.718360,67.488472,6.285718,67.488472,996
997,8.499586,2.137927,8.499586,62.427025,5.716910,62.427025,997
998,9.474313,2.185626,9.474313,64.107925,5.646301,64.107925,998
999,8.938540,2.089374,8.938540,66.466278,5.989630,66.466278,999


----------------------------------------


AttributeError: module 'tensorflow' has no attribute 'random_uniform'